<a href="https://colab.research.google.com/github/flohmannjr/tensorflow_curso/blob/main/TensorFlow_TL4_Outros_Modelos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# TensorFlow Tranfer Learning Teste de Modelos

In [ ]:
import tensorflow as tf
import tensorflow_hub as hub

import matplotlib.pyplot as plt
import seaborn as sns

import re

from tensorflow.keras import Model, Sequential
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Input
from tensorflow.keras.layers import Rescaling, RandomFlip, RandomHeight, RandomRotation, RandomWidth, RandomZoom
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import image_dataset_from_directory

from tensorflow.keras.applications import EfficientNetB0, EfficientNetB1, EfficientNetB2, EfficientNetB3, InceptionV3, ResNet50V2

## Setup

In [ ]:
plt.rcParams['figure.figsize'] = [8, 5]
plt.rcParams['figure.dpi'] = 100
plt.style.use('seaborn-darkgrid')

## Constantes

In [ ]:
SEMENTE = 2008193

DIR_TREINO = '10_food_classes_10_percent/train'
DIR_TESTE  = '10_food_classes_10_percent/test'

TAMANHO_LOTE = 32
FORMATO_IMAGEM = (224, 224)
MODO_CLASSE = 'categorical'

FORMATO_ENTRADA = FORMATO_IMAGEM + (3,)
ATIVACAO = 'softmax'

PERDA = 'categorical_crossentropy'
METRICAS = ['accuracy']

# OTIMIZADOR = 'Adam'
APRENDIZADO = 0.001

ITERACOES = 5

### Modelos

In [ ]:
MODELOS_BASE = [EfficientNetB0, EfficientNetB1, EfficientNetB2, EfficientNetB3, InceptionV3, ResNet50V2]

MODELOS_HUB = ['https://tfhub.dev/tensorflow/efficientnet/b0/feature-vector/1',
               'https://tfhub.dev/tensorflow/efficientnet/b1/feature-vector/1',
               'https://tfhub.dev/tensorflow/efficientnet/b2/feature-vector/1',
               'https://tfhub.dev/tensorflow/efficientnet/b3/feature-vector/1',
               'https://tfhub.dev/google/imagenet/resnet_v1_50/feature_vector/5',
               'https://tfhub.dev/google/imagenet/resnet_v2_50/feature_vector/5',
               'https://tfhub.dev/google/imagenet/inception_v3/feature_vector/5']

# Modelos testados e descartados:
# Com performance muito inferior: ResNet50, VGG16
# Com performance levemente inferior:
# EfficientNetB4, EfficientNetB5, EfficientNetB6, EfficientNetB7
# EfficientNetV2B0, EfficientNetV2B1, EfficientNetV2B2, EfficientNetV2B3
# https://tfhub.dev/tensorflow/efficientnet/b4/feature-vector/1
# https://tfhub.dev/tensorflow/efficientnet/b5/feature-vector/1
# https://tfhub.dev/tensorflow/efficientnet/b6/feature-vector/1
# https://tfhub.dev/tensorflow/efficientnet/b7/feature-vector/1

## Funções

In [ ]:
!wget https://raw.githubusercontent.com/flohmannjr/tensorflow_curso/main/funcoes.py

In [ ]:
from funcoes import criar_callback_tensorboard, grafico_historico_por_iteracao, unzip_data

## Dados

In [ ]:
!wget https://storage.googleapis.com/ztm_tf_course/food_vision/10_food_classes_10_percent.zip

unzip_data('10_food_classes_10_percent.zip')

### Importando os dados e tranformando-os em lotes

In [ ]:
dados_treino = image_dataset_from_directory(directory=DIR_TREINO,
                                            batch_size=TAMANHO_LOTE,
                                            image_size=FORMATO_IMAGEM,
                                            label_mode=MODO_CLASSE,
                                            seed=SEMENTE)

dados_teste = image_dataset_from_directory(directory=DIR_TESTE,
                                           batch_size=TAMANHO_LOTE,
                                           image_size=FORMATO_IMAGEM,
                                           label_mode=MODO_CLASSE,
                                           shuffle=False, # Não embaralhar os dados de teste para permitir avaliarmos as previsões.
                                           seed=SEMENTE)

rotulos = dados_treino.class_names

## Modelos

In [ ]:
modelos = {}

In [ ]:
for mb in MODELOS_BASE:
    modelo_base = mb(include_top=False)
    modelo_base.trainable = False

    modelo_nome = f'modelo_base_{modelo_base.name}'

    entradas = Input(shape=FORMATO_ENTRADA, name='camada_entrada')

    expansao_dados = Sequential(name='expansao_dados')

    if modelo_base.name[:12] != 'efficientnet':
        expansao_dados.add(Rescaling(1./255))

    expansao_dados.add(RandomFlip('horizontal'))
    expansao_dados.add(RandomHeight(0.2))
    expansao_dados.add(RandomRotation(0.2))
    expansao_dados.add(RandomWidth(0.2))
    expansao_dados.add(RandomZoom(0.2))

    # Há um bug na versão 2.8 do TensorFlow que faz necessário forçar o treinamento para que a expansão dos dados funcione.
    expandidos = expansao_dados(entradas, training=True)

    camadas = modelo_base(expandidos, training=False)
    camadas = GlobalAveragePooling2D(name='agrupamento_media_global')(camadas)

    saidas = Dense(len(rotulos), activation=ATIVACAO, name='camada_saida')(camadas)

    modelo = Model(inputs=entradas, outputs=saidas, name=modelo_nome)

    modelo.compile(loss=PERDA,
                   optimizer=Adam(learning_rate=APRENDIZADO),
                   metrics=METRICAS)
    
    historico = modelo.fit(dados_treino,
                           epochs=ITERACOES,
                           steps_per_epoch=len(dados_treino),
                           validation_data=dados_teste,
                           validation_steps=len(dados_teste),
                           callbacks=[criar_callback_tensorboard(diretorio='outros_modelos', experimento=modelo_nome)],
                           verbose=1)

    modelos[modelo_nome] = modelo
    
    grafico_historico_por_iteracao(historico)


for mh in MODELOS_HUB:
    modelo_nome = "modelo_hub_" + re.sub('\W+', '_', mh[18:])
    
    modelo = Sequential(name=modelo_nome)

    modelo.add(Rescaling(1./255))
    modelo.add(RandomFlip('horizontal'))
    modelo.add(RandomHeight(0.2))
    modelo.add(RandomRotation(0.2))
    modelo.add(RandomWidth(0.2))
    modelo.add(RandomZoom(0.2))

    modelo.add(hub.KerasLayer(handle=mh,
                              trainable=False,
                              input_shape=FORMATO_ENTRADA))

    modelo.add(Dense(len(rotulos), activation=ATIVACAO))

    modelo.compile(loss=PERDA,
                   optimizer=Adam(learning_rate=APRENDIZADO),
                   metrics=METRICAS)
    
    historico = modelo.fit(dados_treino,
                           epochs=ITERACOES,
                           steps_per_epoch=len(dados_treino),
                           validation_data=dados_teste,
                           validation_steps=len(dados_teste),
                           callbacks=[criar_callback_tensorboard(diretorio='outros_modelos', experimento=modelo_nome)],
                           verbose=1)

    modelos[modelo_nome] = modelo
    
    grafico_historico_por_iteracao(historico)


## Salvando modelos

In [ ]:
# for modelo_nome, modelo in modelos:
#     modelo.save(f'{modelo_nome}_HDF5.h5')

## Comparando modelos utilizando TensorBoard

In [ ]:
# !tensorboard dev upload --logdir ./outros_modelos/ \
#                         --name "Teste de Modelos." \
#                         --description "Teste de modelos EfficientNet, Inception, ResNet. Base de dados Food101 reduzida." \
#                         --one_shot

TensorBoard link: 

In [ ]:
# Apagar diretórios não-vazios
# import shutil
# shutil.rmtree('outros_modelos')